In [ ]:
!pip install PyCryptoDomex elgamal pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.0 MB/s eta 0:00:00


In [ ]:
import random
from Cryptodome.PublicKey import RSA
from Cryptodome.Cipher import PKCS1_OAEP
from elgamal.elgamal import Elgamal

import time
from secrets import token_bytes
import warnings


In [ ]:
def modular_inverse(a, mod):
    b0, x0, x1 = mod, 0, 1
    while a > 1:
        q = a // b0
        a, b0 = b0, a % b0
        x0, x1 = x1 - q * x0, x0
    return x1 + mod if x1 < 0 else x1

def select_prime_larger_than(value):
    mersenne_primes = [2**x - 1 for x in [17, 19, 31, 61, 89, 107, 127, 521]]
    for prime in mersenne_primes:
        if prime > value:
            return prime
    return 2**128 + 51

def split_secret(secret_bytes, required_shares, distributed_shares, prime_mod=None):

    secret = int.from_bytes(secret_bytes, byteorder='big', signed=False)
    prime_mod = prime_mod or select_prime_larger_than(secret)

    coefficients = [int.from_bytes(token_bytes((prime_mod.bit_length() + 7) // 8), byteorder='big', signed=False) % prime_mod for _ in range(required_shares - 1)]
    coefficients.append(secret)

    shares = [(i, (evaluate_polynomial(i, coefficients, prime_mod)).to_bytes((prime_mod.bit_length() + 7) // 8, byteorder='big', signed=False)) for i in range(1, distributed_shares + 1)]
    return {"required_shares": required_shares, "prime_mod": prime_mod.to_bytes((prime_mod.bit_length() + 7) // 8, byteorder='big', signed=False), "shares": shares}

def evaluate_polynomial(x, coefficients, prime_mod):
    y = 0
    for coefficient in coefficients:
        y = (y * x + coefficient) % prime_mod
    return y

def lagrange_interpolation(x, points, prime_mod):
    y = 0
    for i, (xi, yi) in enumerate(points):
        numerator = yi
        denominator = 1
        for j, (xj, yj) in enumerate(points):
            if j != i:
                numerator = (numerator * (x - xj)) % prime_mod
                denominator = (denominator * (xi - xj)) % prime_mod
        y = (y + numerator * modular_inverse(denominator, prime_mod)) % prime_mod
    return y

def recover_secret(data):
    shares = data['shares']
    required_shares = data['required_shares']
    prime_mod = int.from_bytes(data['prime_mod'], byteorder='big', signed=False)

    if len(shares) < required_shares:
        raise ValueError("not enough shares provided")
    shares = shares[:required_shares]

    points = [(x, int.from_bytes(y, byteorder='big', signed=False)) for x, y in shares]
    return (lagrange_interpolation(0, points, prime_mod)).to_bytes((prime_mod.bit_length() + 7) // 8, byteorder='big', signed=False)[1:]

In [ ]:
secret = b"This is my secret"
required_shares = 3
total_shares = 5

split_data = split_secret(secret, required_shares, total_shares)
split_data_rsa = split_data['shares'][:3]
split_data_ecc = split_data['shares'][2:]

print("Original Secret (for splitting):", secret)

Original Secret (for splitting): b'This is my secret'


In [ ]:
rsa_private_key = RSA.generate(1024)
rsa_public_key = rsa_private_key.publickey()
rsa_cipher_encrypt = PKCS1_OAEP.new(rsa_public_key)
rsa_cipher_decrypt = PKCS1_OAEP.new(rsa_private_key)


rsa_encrypted_shares = []
start_time = time.time()
for share in split_data_rsa:
    encrypted_share = rsa_cipher_encrypt.encrypt(share[1])
    rsa_encrypted_shares.append((share[0], encrypted_share))
rsa_time = time.time() - start_time


print("\nRSA Encrypted Shares:")
for idx, encrypted_share in rsa_encrypted_shares:
    print(f"Part {idx}: {encrypted_share}")


rsa_decrypted_shares = []
start_time = time.time()
for idx, encrypted_share in rsa_encrypted_shares:
    decrypted_share = rsa_cipher_decrypt.decrypt(encrypted_share)
    rsa_decrypted_shares.append((idx, decrypted_share))
rsa_decrypt_time = time.time() - start_time


print("\nRSA Decrypted Shares:")
for idx, decrypted_share in rsa_decrypted_shares:
    print(f"Part {idx}: {decrypted_share}")


recovery_data_rsa = {
    'required_shares': required_shares,
    'prime_mod': split_data['prime_mod'],
    'shares': rsa_decrypted_shares
}
reconstructed_secret_rsa = recover_secret(recovery_data_rsa)

print(f"\nReconstructed Secret (RSA): {reconstructed_secret_rsa.decode('utf-8')}")
print(f"\nRSA Encryption Time: {rsa_time:.4f} seconds")
print(f"RSA Decryption Time: {rsa_decrypt_time:.4f} seconds")


RSA Encrypted Shares:
Part 1: b'\xbc\x9f2U\xf4\xa1-\x13\xc0\xf9\x96X\xe4\x1b\xeeby\xa1\x8e\xd1.$\xa9_\x92\x99\xf5\xe6p-\xf4\x11\\6\x1c\x94\\\x8c\x9c]\x89\x94-\xa9\x82\xd2<\xfb\xb2 \x94l&\x85\x8cx\xce\x80-\x9b.\x05\x97\x18\x1c\x8a\xe0\x06\x0cz\xb9\x19\xf2*\x9e\x03\xf2c\x9a\xd6&\x120\xc7UO\x89@s\x1e\\h\xf1}\xae\xc4R\xab\x85\x7fW\xa4o7\xbcD\xdb\xb7\xf0U\xd6\xa9\x15qu\x8f\xd3\xc2\x0c\xd2\x00\xac\x82\x8dU\x85]>'
Part 2: b"\x00%\xfdI\xca;\xe8\xe7\xe0\xb0(\xd0\xd7?9]\x8auhqm\xf9\x94\xab,\xa2my\xdb\xba\xaa\xa2\xfbT\xa5\x97\xa11\xb5L}\xb7\xd0\x10@v\x07\xaeLt\xc0W\x9bRmB\xac$\xaf\x92\xf0e\xe8\xa0*{\x0fA\x13j\x85\x81\xda\x0b\x0b\x12|\n\x85t\xd2\xb3\x98KLo\x82\xf3\x1a\x14\xf2\x8cN\xed\x80F\xc7)L\x84\x0f\x80\tZ\xad\x02\xdc\xdey1\xf8#\xcb\x16\xfbI\xbf\xc5\xe3s[L'\xff\xd4?\x12E"
Part 3: b'\xc3\x87[\xcc\xeb\xe5H\x9e\xa2\xcdv\x8e\xd1\xe2\x8a\xfag\xc8\x86g>b\ry\xa1\xe6\xe6\xfd\xd7\xf8H\x13A\xed\xb9\xb8Kx\xbc\xf9\xb1\x7f-\xaa\xd8\x9dr\xa4KX\x90c\xa7~D\xb7\xcb\xee\x7f\x81\xc7\xe9\x86\xd5=\xe3F\xfa\xfe\xd

In [ ]:
elgamal_pb, elgamal_pv = Elgamal.newkeys(128)

elgamal_encrypted_shares = []
start_time = time.time()
for share in split_data_ecc:
    encrypted_share = Elgamal.encrypt(share[1], elgamal_pb)
    elgamal_encrypted_shares.append((share[0], encrypted_share))
elgamal_time = time.time() - start_time

print("\nElGamal Encrypted Shares:")
for idx, encrypted_share in elgamal_encrypted_shares:
    print(f"Part {idx}: {encrypted_share}")


elgamal_decrypted_shares = []
start_time = time.time()
for idx, encrypted_share in elgamal_encrypted_shares:
    decrypted_share = Elgamal.decrypt(encrypted_share, elgamal_pv)
    decrypted_share_bytes = bytes(decrypted_share)
    elgamal_decrypted_shares.append((idx, decrypted_share_bytes))
elgamal_decrypt_time = time.time() - start_time

print("\nElGamal Decrypted Shares:")
for idx, decrypted_share in elgamal_decrypted_shares:
    print(f"Part {idx}: {decrypted_share}")



recovery_data_ecc = {
    'required_shares': required_shares,
    'prime_mod': split_data['prime_mod'],
    'shares': elgamal_decrypted_shares
}
reconstructed_secret_ecc = recover_secret(recovery_data_ecc)

print(f"Reconstructed Secret (ElGamal): {reconstructed_secret_ecc.decode('utf-8')}")
print(f"ElGamal Encryption Time: {elgamal_time:.4f} seconds")
print(f"ElGamal Decryption Time: {elgamal_decrypt_time:.4f} seconds")



ElGamal Encrypted Shares:
Part 3: CipherText(13682021647660526334453413078523400064688192645543360219462946423300198264687221669969730621769869999677609783604327004983832916497657522965940585012416878613914120856922588509006734062112033499640372919908166399954314489968689534400953427799890447936320776354170449056181236999546973550040450040249493677778675020013682267899986375279646508499356336991884253494352539112885503782548103085301417418603365426374596331659494437890102694786269141485713832612261791585032287417822725359368575726412642218067846854285593933833927420716877598385230608979043707176131936210443889725123306273641760728710131763530591281019740, 48519663039237059503708134624316263950597134417612960149646560550192916480450019357446229466980192088590046167185412695105173108690562965231105497340814500608339242704453143044791301325297747765081758497461095885350938098900544885225863183761854456234839401014082397287296379777599851026251625262345038760992332901680928063430930601024